In [3]:
# Linear algebra
import scipy.stats as ss
import scipy.special
from scipy import optimize
from mpmath import gamma
import numpy as np
import scipy
from scipy.stats import norm
from numpy import linalg as la
from scipy import sparse
from scipy.sparse.linalg import spsolve
import pandas as pd
from scipy.stats import multivariate_normal
#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
### plotting
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D
import cvxpy as cp
import time
### Other
import itertools
from timeit import default_timer as timer
from itertools import product
from ipywidgets import interact, widgets
import warnings
warnings.filterwarnings("ignore")

In [5]:
def LagQUAD_fourier_NIG_call_on_min_pricer(S0,K,r,T,alpha,beta,delta,DELTA,N,R):
  """Compute rainbow options under the NIG model using Gauss-Laguerre quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - theta (array): Array of skewness parameters of each stock.
  - nu (float): controls the kurtosis of the distribution of log-returns
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Laguerre quadrature.
  - R (array): Array of damping parameters.
  Returns:
  - V (float): Option price estimate
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.laguerre.laggauss(N)  # Laguerre quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = list(product(*u_1d))  # Full Tensor Isotropic Grid of abcissas
  w_ft = list(product(*w_1d))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(len(u_ft)):  # For all grid points
    u_list = laguerre_combinations(u_ft[i])  # Get combinations of Laguerre terms
    y_list = u_list + 1j * R
    phi_list = np.zeros(len(u_list), dtype=complex)  # List of characteristic function evaluations
    p_list = np.zeros(len(u_list), dtype=complex)  # List of payoff function evaluations
    g_list = np.zeros(len(u_list), dtype=complex)  # List of integrands resulting from change of variables
    for k in range(len(u_list)):  # For all possible combinations within 1 grid point
      phi_list[k] = NIG_characteristic_function(y_list[k], T, r, alpha, beta, delta, DELTA)
      p_list[k] = fourier_payoff_call_on_min(y_list[k])
      g_list[k] = np.exp(np.multiply(1j, np.dot(u_list[k], X0))) * phi_list[k] * p_list[k]
    g = np.sum(g_list)
    V = V + discount * np.exp(np.sum(u_ft[i])) * np.prod(w_ft[i]) * g
  return np.real(V)  # Real Part of the integral

def LagQUAD_fourier_NIG_basket_put_pricer (S0,K,r,T,alpha,beta,delta,DELTA,N,R):
  """Compute rainbow options under the NIG model using Gauss-Laguerre quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - theta (array): Array of skewness parameters of each stock.
  - nu (float): controls the kurtosis of the distribution of log-returns
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Laguerre quadrature.
  - R (array): Array of damping parameters.
  Returns:
  - V (float): Option price estimate
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.laguerre.laggauss(N)  # Laguerre quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = list(product(*u_1d))  # Full Tensor Isotropic Grid of abcissas
  w_ft = list(product(*w_1d))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, d*K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(len(u_ft)):  # For all grid points
    u_list = laguerre_combinations(u_ft[i])  # Get combinations of Laguerre terms
    y_list = u_list + 1j * R
    phi_list = np.zeros(len(u_list), dtype=complex)  # array of characteristic function evaluations
    p_list = np.zeros(len(u_list), dtype=complex)  # array of payoff function evaluations
    g_list = np.zeros(len(u_list), dtype=complex)  # array of integrands resulting from the change of variables
    for k in range(len(u_list)):  # For all possible combinations within 1 grid point
      phi_list[k] = NIG_characteristic_function(y_list[k], T, r, alpha, beta, delta, DELTA)
      p_list[k] = fourier_payoff_basket_put(y_list[k])
      g_list[k] = np.exp(np.multiply(1j, np.dot(u_list[k], X0))) * phi_list[k] * p_list[k]
    g = np.sum(g_list)
    V = V + discount * np.exp(np.sum(u_ft[i])) * np.prod(w_ft[i]) * g
  return np.real(V)  # Real Part of the integral

def HermQUAD_fourier_NIG_call_on_min_pricer(S0,K,r,T,alpha,beta,delta,DELTA,N,R):
  """Compute rainbow options under the VG model using Gauss-Laguerre quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - theta (array): Array of skewness parameters of each stock.
  - nu (float): controls the kurtosis of the distribution of log-returns
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Laguerre quadrature.
  - R (array): Array of damping parameters.
  Returns:
  - V (float): Option price estimate
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.hermite.hermgauss(N)  # Hermite quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = np.asarray(list(product(*u_1d)))  # Full Tensor Isotropic Grid of abcissas
  w_ft = np.asarray(list(product(*w_1d)))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount_factor = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-R @ X0)  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(u_ft.shape[0]):  # For all grid points
    y = u_ft[i] + 1j * R
    phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)
    payoff = fourier_payoff_call_on_min(y)
    V += np.exp(np.sum(u_ft[i]**2)) * np.prod(w_ft[i]) * np.exp(1j * u_ft[i] @ X0) * phi * payoff
  return np.real(discount_factor * V)  # Real Part of the integral

def HermQUAD_fourier_NIG_basket_put_pricer(S0,K,r,T,alpha,beta,delta,DELTA,N,R):
  """Compute rainbow options under the NIG model using Gauss-Laguerre quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - theta (array): Array of skewness parameters of each stock.
  - nu (float): controls the kurtosis of the distribution of log-returns
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Laguerre quadrature.
  - R (array): Array of damping parameters.
  Returns:
  - V (float): Option price estimate
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.hermite.hermgauss(N)  # Hermite quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = np.asarray(list(product(*u_1d)))  # Full Tensor Isotropic Grid of abcissas
  w_ft = np.asarray(list(product(*w_1d)))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, d*K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount_factor = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-R @ X0)  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(u_ft.shape[0]):  # For all grid points
    y = u_ft[i] + 1j * R
    phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)
    payoff = fourier_payoff_basket_put(y)
    V += np.exp(np.sum(u_ft[i]**2)) * np.prod(w_ft[i]) * np.exp(1j * u_ft[i] @ X0) * phi * payoff
  return np.real(discount_factor * V)  # Real Part of the integral

def covariance_matrix(sigma, rho):
    """Compute the covariance matrix.
    Args:
    - sigma (array): Array of volatilities of each stock.
    - rho (array): Correlation matrix.
    Returns:
    - SIGMA (array): Covariance matrix.
    """
    sigma = np.diag(sigma)  # Diagonal matrix of volatilities
    SIGMA = np.dot(sigma, np.dot(rho, sigma))  # Covariance matrix calculation
    return SIGMA

def NIG_characteristic_function(u, T, r, alpha, beta, delta, DELTA):
  """Compute the characteristic function of the NIG distribution.
  Args:
  - u (array): Vector in R^d.
  - T (float): Terminal time.
  - r (float): Short rate.
  - alpha (float): Alpha parameter of the NIG distribution.
  - beta (array): Array of beta values.
  - delta (float): Delta parameter of the NIG distribution.
  - DELTA (array): Covariance matrix.
  Returns:
  - phi (complex): Characteristic function value.
  """
  mu = -delta * (np.sqrt(alpha**2 - np.square(beta)) - np.sqrt(alpha**2 - np.square(beta + 1)))
  phi = np.exp(np.multiply(1j * T, (r + mu) @ u) + delta * T * (np.sqrt(alpha**2 - beta @ DELTA @ beta) - np.sqrt(alpha**2 - (beta + np.multiply(1j, u)) @ DELTA @ (beta + np.multiply(1j, u)))))
  return phi


def fourier_payoff_call_on_min(u):
    """Compute the Fourier of the payoff of scaled (K = 1) call on min option.
    Args:
    - u (array): Array of Fourier frequencies.
    Returns:
    - payoff (float): Call on min option payoff Fourier transofrm value.
    """
    denominator = (np.multiply(1j, np.sum(u)) - 1) * np.prod(np.multiply(1j, u))
    return 1 / denominator

def fourier_payoff_basket_put(u):
  """Compute the Fourier of the payoff of scaled (K = 1) basket put option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  numerator = np.prod(scipy.special.gamma(np.multiply(-1j,u)))
  denominator = scipy.special.gamma(-1j*(np.sum(u))+2)
  return (numerator/denominator)


def integrand_to_optimize_NIG_call_on_min(R):
    """Calculate the integrand for QMC estimation of the rainbow option under NIG model.
    Args:
    - R (array): Array of damping parameters.
    Returns:
    - integrand (float): Integrand value.
    """
    d = len(S0)  # Dimensionality
    X0 = np.log(np.divide(S0, K))  # Element-wise division
    y = np.multiply(1j, R)
    phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Characteristic function
    p = fourier_payoff_call_on_min(y)  # Fourier Transformed Payoff function
    discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
    integrand = K * discount * phi * p
    return integrand

def integrand_to_optimize_NIG_basket_put(R):
    """Calculate the integrand for QMC estimation of the basket put option under NIG model.
    Args:
    - R (array): Array of damping parameters.
    Returns:
    - integrand (float): Integrand value.
    """
    d = len(S0)  # Dimensionality
    X0 = np.log(np.divide(S0, d*K))  # Element-wise division
    y = np.multiply(1j, R)
    phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Characteristic function
    p = fourier_payoff_basket_put(y)  # Fourier Transformed Payoff function
    discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
    integrand = K * discount * phi * p
    return integrand

def laguerre_combinations(u):
  """Generate combinations of Laguerre terms due to change of variable from (-infinity,+infinity) to (0,+infinity)

  Args:
  - u (array): Laguerre terms.

  Returns:
  - Z (array): Combinations of Laguerre terms.
  """
  d = len(u)  # Dimensionality
  aux = np.array(u) * (-1)  # Negate each element of u
  aux = tuple(aux)  # Convert to tuple
  L = [(u[i], aux[i]) for i in range(len(u))]  # Create pairs of (u[i], -u[i])
  Z = list(itertools.product(*L))  # Generate all possible combinations of the pairs
  return np.array(Z)

# Call on min options

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [6]:
# Payoff parameters
dimension = 5
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def NIG_constraint(R):
  return alpha**2 - (beta- R) @ DELTA @ (beta - R)

def rainbow_constraint_1(R):
  return -1*R

def rainbow_constraint_2(R):
  return -1 - np.sum(R)

cons = ( {'type': 'ineq', 'fun': NIG_constraint},
        {'type': 'ineq', 'fun': rainbow_constraint_1},
        {'type': 'ineq', 'fun': rainbow_constraint_2},)

R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_call_on_min, constraints = cons, x0 = R_init , method = "Trust-Constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [-6.15725122 -6.15725122 -6.15725121 -6.15725121 -6.15725122]


## Pricing using Tensor Product Gauss-Laguerre quadrature in the Fourier space

In [8]:
############### Model and payoff parameters ###############
# Payoff parameters
dimension = 5
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### Laguerre Quadrature parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature nodes per dimension
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_call_on_min, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
LagQUAD_estimate = LagQUAD_fourier_NIG_call_on_min_pricer(S0,K,r,T,alpha,beta,delta,DELTA,N,R)
print("LagQUAD estimate =", round(LagQUAD_estimate,5)  )

LagQUAD estimate = 0.08377


# Basket put options

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [9]:
# Payoff parameters
dimension = 3
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def NIG_constraint(R):
  return alpha**2 - (beta- R) @ DELTA @ (beta - R)

def basket_put_constraint(R):
    return R
cons = ( {'type': 'ineq', 'fun': NIG_constraint},
        {'type': 'ineq', 'fun': basket_put_constraint},)

R_init = 2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_basket_put, constraints = cons, x0 = R_init , method = "Trust-Constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [3.52609666 3.52609798 3.52609763]


In [11]:
############### Model and payoff parameters ###############
# Payoff parameters
dimension = 3
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### Laguerre Quadrature parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature nodes per dimension
R_init = 2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_basket_put, constraints = cons, x0 = R_init , method = "Trust-Constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
LagQUAD_estimate = LagQUAD_fourier_NIG_basket_put_pricer(S0,K,r,T,alpha,beta,delta,DELTA,N,R)
print("LagQUAD estimate =", round(LagQUAD_estimate,5)  )

LagQUAD estimate = 3.32423


# 1D multi-strike vectorized implementations (Caching)

In [12]:
def multistrike_LagQUAD_fourier_NIG_vanilla_call_pricer(S0,K,r,T,alpha,beta,delta,DELTA,N,R):
  # vectorized implementation of the pricer for Vanilla calls.
  K = np.array(K).reshape([len(K),1]) # list of strikes
  d = 1
  [u,w] = np.polynomial.laguerre.laggauss(N) #Laguerre quadrature nodes and weights.
  u_1d=[u for i in range(d)] #array of 1-D abcissas for all directions
  w_1d=[w for i in range(d)] #array of 1-D weights for all directions
  u = np.asarray(list(product(*u_1d))) # Full Tensor Isotropic Grid of abcissas
  w = np.asarray(list(product(*w_1d))) #Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0,K)) #element-wise division X_0 is logarithm of stock price at initial time
  phi_list = np.zeros(N, dtype = complex) #List of characteristic function evaluations
  p_list = np.zeros(N, dtype = complex) #List of payoff function evaluations
  w_prod_list = np.zeros(N, dtype = float)
  reciprocal_weight_function = np.zeros(N, dtype = float)
  phi_values = np.array(list(map(lambda u: NIG_characteristic_function(u + 1j*R, T, r, alpha, beta, delta, DELTA), u) ) )
  p_values = np.array(list(map(fourier_payoff_call_on_min, u+1j*R)))
  w_prod_values = np.prod(w,axis = 1)
  reciprocal_weight_function = np.exp(np.sum(u,axis = 1 ))
  temp = p_values * phi_values * w_prod_values * reciprocal_weight_function
  mat = np.exp(1j * np.outer(X0,u))
  K_factor = (2*np.pi)**(-d) * np.exp(-r*T) * K * np.exp(-R*X0)
  V = 2 * K_factor.reshape(K.shape[0],)  * np.real( mat @ temp ) # The factor 2 comes from evenness of the integrand
  return V

In [22]:
# Payoff parameters
K_grid = list(np.linspace(80,120,40)) # strike prices grid
dimension = 1
S0 = 100 * np.ones(dimension)
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def NIG_constraint(R):
  return alpha**2 - (beta- R) @ DELTA @ (beta - R)

def rainbow_constraint_1(R):
  return -1*R

def rainbow_constraint_2(R):
  return -1 - np.sum(R)

cons = ( {'type': 'ineq', 'fun': NIG_constraint},
        {'type': 'ineq', 'fun': rainbow_constraint_1},
        {'type': 'ineq', 'fun': rainbow_constraint_2},)

R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
K = np.median(K_grid)
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_call_on_min, constraints = cons, x0 = R_init , method = "Trust-Constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

############### Laguerre Quadrature parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature points per dimension.
LagQUAD_estimates =  multistrike_LagQUAD_fourier_NIG_vanilla_call_pricer(S0,K_grid,r,T,alpha,beta,delta,DELTA,N,R)
print("LagQUAD estimates =", LagQUAD_estimates )

Optimal damping parameters: [-10.45882459]
LagQUAD estimates = [20.42050621 19.44759116 18.49031262 17.54954791 16.6255617  15.71835665
 14.8279665  13.95467003 13.09911988 12.26239188 11.44596738 10.65166496
  9.88153857  9.1377584   8.42248811  7.73776955  7.08542271  6.46696571
  5.88355738  5.33596237  4.82453764  4.34923771  3.90963548  3.50495532
  3.13411482  2.795772    2.48837514  2.21021266  1.95946118  1.73423022
  1.53260241  1.35266869  1.19255809  1.05046207  0.92465358  0.81350131
  0.71547942  0.62917338  0.55328252  0.48661965]


# Multidimensional multistrike vectorized implementation (caching)

In [15]:
def phase_factor(u,X0):
  return np.exp(1j * u @ X0.T )

def multistrike_LagQUAD_fourier_NIG_call_on_min_pricer(S0,K,r,T,alpha,beta,delta,DELTA,N,R):
  d=len(S0) #number of underlying stocks / dimensionality of the problem
  K = np.array(K).reshape([len(K),1]) # list of strikes to be valued simulatenously
  k = K.shape[0] # number of strikes to be valued
  [u,w] = np.polynomial.laguerre.laggauss(N) #returns Laguerre quadrature nodes and weights.
  u_1d=[u for i in range(d)] # Nxd array of 1D abcissas
  w_1d=[w for i in range(d)] # Nxd array of 1D weights
  u = np.asarray(list(product(*u_1d))) # N^d x d: array of N^d d-dimensional points
  w = np.asarray(list(product(*w_1d))) # N^d x d: array of N^d d-dimensional weights
  u_laguerre_scaling = np.repeat(a = u, repeats = 2**d ,axis = 0) # Repeats each element of the array 2**d times before moving to next element
  w = np.repeat(a = w, repeats = 2**d ,axis = 0) # Repeats each element of the array 2**d times before moving to next element
  u = np.vstack(np.array(list(map(laguerre_combinations, u)))) # creates all possible combinations of multiplying by -1.
  X0 = np.log(np.divide(S0,K)) #element-wise division X_0 is logarithm of stock price at initial time - Division by d is intrinsic to basket option for equally weighted average
  phi_values =  np.array(list(map(lambda u: NIG_characteristic_function(u + 1j*R, T, r, alpha, beta, delta, DELTA), u) ), dtype = complex ) # characteristic function evaluations
  p_values = np.array(list(map(fourier_payoff_call_on_min, u+1j*R)), dtype = complex) # payoff transform evaluations
  w_prod_values = np.prod(w,axis = 1) # contains product of weights for each multi-index
  reciprocal_weight_function = np.exp(np.sum(u_laguerre_scaling,axis = 1 ))  # contains the inverse of the weight function applied to use Gauss-Laguerre.
  temp =  p_values * phi_values * w_prod_values * reciprocal_weight_function
  mat = np.array(list(map(lambda u: phase_factor(u,X0), u)))
  K_factor = (2*np.pi)**(-d) * np.exp(-r*T) * K *  np.exp(-1*np.einsum("ij,ij->i", np.tile(R,(k,1)), X0)).reshape(k,1) # Einstein sumation allows for dot product row-by-row between two matrices. Tiling repeats the vector k times, axis = 1 is to make sure we copy rows.
  V = K_factor * np.real(np.einsum("ij,ij->i", mat.T, np.tile(temp,(k,1)))).reshape(k,1)
  return V

In [23]:
# Payoff parameters
K_grid = list(np.linspace(80,120,40)) # strike prices grid
dimension = 3
S0 = 100 * np.ones(dimension)
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### Setting for the optimal damping parameters #############
# Constraint related to the strip of regularity of the extended characteristic function
def NIG_constraint(R):
  return alpha**2 - (beta- R) @ DELTA @ (beta - R)

# Constraints related to the strip of regularity of the payoff transform
def rainbow_constraint_1(R):
  return -1*R

def rainbow_constraint_2(R):
  return -1 - np.sum(R)

cons = ( {'type': 'ineq', 'fun': NIG_constraint},
        {'type': 'ineq', 'fun': rainbow_constraint_1},
        {'type': 'ineq', 'fun': rainbow_constraint_2},)

R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
K = np.median(K_grid)
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_call_on_min, constraints = cons, x0 = R_init , method = "Trust-Constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

############### Laguerre Quadrature parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature points per dimension.
LagQUAD_estimates =  multistrike_LagQUAD_fourier_NIG_call_on_min_pricer(S0,K_grid,r,T,alpha,beta,delta,DELTA,N,R)
print("LagQUAD estimates =", LagQUAD_estimates )

Optimal damping parameters: [-8.0741749  -8.07417437 -8.07471346]
LagQUAD estimates = [[1.04401497e+01]
 [9.59350277e+00]
 [8.78030583e+00]
 [8.00079116e+00]
 [7.25480160e+00]
 [6.54216246e+00]
 [5.86293854e+00]
 [5.21760041e+00]
 [4.60710706e+00]
 [4.03289971e+00]
 [3.49679956e+00]
 [3.00081241e+00]
 [2.54686128e+00]
 [2.13648634e+00]
 [1.77056202e+00]
 [1.44908027e+00]
 [1.17103565e+00]
 [9.34427298e-01]
 [7.36370566e-01]
 [5.73292768e-01]
 [4.41177272e-01]
 [3.35818285e-01]
 [2.53054155e-01]
 [1.88956823e-01]
 [1.39966270e-01]
 [1.02968858e-01]
 [7.53259185e-02]
 [5.48632712e-02]
 [3.98337744e-02]
 [2.88643241e-02]
 [2.08966941e-02]
 [1.51290736e-02]
 [1.09626744e-02]
 [7.95567422e-03]
 [5.78520653e-03]
 [4.21707664e-03]
 [3.08231347e-03]
 [2.25942781e-03]
 [1.66123330e-03]
 [1.22519545e-03]]
